In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import json
import torch
import pandas as pd
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import os
import skimage.io as io
import glob
from torch_geometric.data import InMemoryDataset, download_url
from torchtext.utils import extract_archive

class Custom_Dataset(torch.utils.data.Dataset):
    
    def __init__(self, metadata_file, img_dir, transform=None, target_transform=None):
        
        
        self.metadata = pd.read_json(metadata_file) 
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
            
    def __len__(self):
        pass

    def __getitem__(self, dataset, idx):
        
        img_name = os.path.join(self.extracted+"/"+self.metadata.resources[0]["name"],
                                [os.path.basename(x) for x in glob.glob(self.extracted+"/"+self.metadata.resources[0]["name"] + "/*")][idx])
        
        image = io.imread(img_name)
        
        label = dataset[idx][1]
       
        sample = {'image': image, 'label': label}
        
        fig = plt.figure()
        plt.imshow(image, cmap='gray', interpolation='none')
        plt.title("Image: {}".format(idx))
            
        return sample, fig
    
    def __download_data__(self):
        download_url(self.metadata["resources"][0]["path"], self.img_dir, log=True)
        
    def __extract_data__(self, cDir, eDir):
        extract_archive(cDir, eDir) #overwrite=True
        self.extracted = eDir
    
    def __dataloader__(self, batch_size, shuffle=True):
        self.dataset = datasets.ImageFolder(os.path.abspath(self.img_dir+"/.."), transform=self.transform)
        self.training_generator = torch.utils.data.DataLoader(self.dataset, batch_size=batch_size, shuffle=shuffle)